In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.size': 17,         # Grundschriftgröße
    'axes.titlesize': 20,    # Titelgröße
    'axes.labelsize': 17,    # Achsentitel
    'legend.fontsize': 17,   # Legenden
    'xtick.labelsize': 17,
    'ytick.labelsize': 17,
    'lines.linewidth': 3.5     # Linienstärke
})
from math import factorial
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore")


# === TRANSFORMED CONSTANTS ===
K = 1e9                  # cells/mm³ (carrying capacity)
r = 0.05                 # min⁻¹ (growth rate)
k_NP = 5.56e-4           # min⁻¹ (neutrophil kill rate)
k_AM = 4.09e-5           # min⁻¹ (macrophage kill rate)
d_N = 6.94e-5            # min⁻¹ (natural E. coli decay)
k_onAM = 1.7458e22       # amount/mm³min (AM binding rate)    #originally: 2.9e4 M⁻¹s⁻¹
k_offAM = 4.2            # min⁻¹ (AM unbinding rate)   # originally: 0.07 s⁻¹
M_max = 4                # cells/mm³ (macrophages)
NP_max = 1.5e4           # cells/mm³ (neutrophils)
EC50_NP = 0.0034         # M (NP activation EC50)
EC50_AM = 0.0056         # M (AM activation EC50)
h_NP = 1.25              # Hill coefficient (neutrophils)
h_AM = 3.16              # HillHill coefficient (macrophages)
t_NFKB_half = 30         # min (NF-κB half-life)
T_c_half = 21            # min (cytokine half-life)
promotor_total = 1e-6    # amount/mm^3 (total promoter concentration), assuming 1 promoter/cell
k1 = 1e-2 * 60           # amount min⁻¹ (NF-κB activation rate),  originally: 1e6 M^-1min^-1
n = 15                   # Shape parameter for gamma term
t_delay=0               # delayed immune response

# Zwei Output-Widgets für die Plots
out1 = widgets.Output()
out2 = widgets.Output()

def run_simulation(N0, k_NP, k_AM, M_max, EC50_NP, T_c_half, t_delay, ylim_linear, xlim, NP_max):
    with out1:
        clear_output(wait=True)
    with out2:
        clear_output(wait=True)

    # Convert EC50 to molecules/mm³ (assuming 1 nM ≈ 6.022e11 molecules/mm³)
    EC50_NP_molecules = EC50_NP * 6.022e11
    EC50_AM_molecules = EC50_AM * 6.022e11

    def model(t, y):
        N, AM_diff, NFKB, C = y  # Unpack variables

        # --- ALGEBRAIC EQUATIONS ---
        # neutrophils
        NP = NP_max * (C**h_NP) / (EC50_NP_molecules**h_NP + C**h_NP)

        # activated macrophages through cytokines
        AM_alg = M_max * (C**h_AM) / (EC50_AM_molecules**h_AM + C**h_AM)

        # total activated macrophages
        AM_combined = (AM_diff + AM_alg) / 2

        # delta
        delta = k_NP * NP + k_AM * AM_combined + d_N * N

        # --- DIFFERENTIAL EQUATIONS ---
        # E.coli growth curve
        dNdt = r * N * (1 - N / K) - delta

        # d[AM]/dt (binding kinetics)
        dAM_diffdt = k_onAM * 2000000 * N * (M_max - AM_diff) - k_offAM * AM_diff
        # The factor 2000000 here represents amount of LPS antigen recognition sites per E.coli cell.
        # It does not appear elsewhere because only AM interacts with LPS. Other immune cells directly influence the cell count.

        # CORRECTED d[NFKB]/dt (gamma-distributed activation)
        gamma_term = (k1 * (t-t_delay))**n / factorial(n - 1) * np.exp(-k1 * (t-t_delay)) if t > t_delay else 0
        dNFKBdt = k1 * (AM_combined - NFKB) * gamma_term - NFKB / t_NFKB_half

        # d[C]/dt cytokine dynamics
        dCdt = 1000 * NFKB / promotor_total - C / T_c_half
        # Assuming one NfKB produces 500 proteins (range: 100-1000)

        return [dNdt, dAM_diffdt, dNFKBdt, dCdt]

    # Initial conditions [N, AM_diff, NFKB, C] - arbitrary
    y0 = [N0, 10, 0.05, 0.1]

    # Time span (0 to 1000 minutes)
    t_span = (0, 1000)
    t_eval = np.linspace(0, 1000, 1000)

    # Solve ODEs
    sol = solve_ivp(model, t_span, y0, t_eval=t_eval, method='BDF', rtol=1e-6, atol=1e-8)

    # Extract solutions
    N = sol.y[0]
    AM_diff = sol.y[1]
    NFKB = sol.y[2]
    C = sol.y[3]

    # Compute algebraic variables
    AM_alg = M_max * (C**h_AM) / (EC50_AM_molecules**h_AM + C**h_AM)
    AM_combined = (AM_diff + AM_alg)
    NP = NP_max * (C**h_NP) / (EC50_NP_molecules**h_NP + C**h_NP)
    delta = k_NP * NP + k_AM * AM_combined + d_N * N

 # Erster (linearer) Plot in out1:
    with out1:
        plt.figure(figsize=(24, 8), facecolor='whitesmoke')
        plt.plot(sol.t, np.maximum(0, N), label='E.coli')
        plt.plot(sol.t, np.maximum(0, AM_combined), label='activated macrophages')
        plt.plot(sol.t, np.maximum(0, NP / 1e3), label='Neutrophils (×10^-3)')
        plt.plot(sol.t, np.maximum(0, C / 1e10), label='Cytokines (x10^-10)')
        plt.plot(sol.t, np.maximum(0, NFKB), label='NFKB')
        plt.plot(sol.t, np.maximum(0, delta), label='delta(cells/μl/min)')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Concentration (amount/μl)')
        plt.title('E. coli-Immune System Dynamics')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, ylim_linear)
        plt.xlim(0, xlim)
        plt.show()

# Globales CSS für Slider und Layout (hier erhöhen wir Schriftart und machen die Slider länger)
display(HTML('''
<style>
    * Abstand zwischen Plot und Slider-Container */
    .slider-container {
        margin-top: 200px;
        margin-bottom: 20px;
    }
    /* Jede Slider-Reihe in zwei Spalten */
    .slider-row {
        display: flex;
        justify-content: space-between;
        margin-bottom: 15px;
    }
    /* Mindestbreite für die Beschriftung */
    .widget-label {
         min-width: 200px !important;
         white-space: nowrap !important;
    }
    /* Layout der Slider: mehr Breite und größere Höhe */
    .slider-box {
        width: 10%;
        height: 50px;
    }
    /* Der Readout (Zahl) soll rechts neben dem Slider erscheinen */
    .widget-inline-hbox {
        display: inline-flex;
        align-items: center;
    }
    .widget-inline-hbox .widget-readout {
        margin-left: 10px !important;
        font-size: 16pt !important;
    }
    /* Erhöhe die Schriftgröße der Slider-Beschriftung */
    .widget-label {
        font-size: 16pt !important;
    }
</style>
'''))

# Erstelle die Parameter-Widgets mit größerem Layout und Schriftart:
slider_style = {'description_width': '400px', 'description': 'font-size: 16pt;'}
slider_box_layout = widgets.Layout(width='200%', height='50px')

w_N0 = widgets.FloatSlider(value=29, min=1, max=100, step=1,
                           description='Initial E. coli (cells/μl)', style=slider_style, layout=slider_box_layout)

w_k_NP = widgets.FloatSlider(value=5.56e-4, min=1e-5, max=1e-3, step=1e-5,
                             description='Kill rate neutrophils (min⁻¹)', style=slider_style, layout=slider_box_layout)

w_k_AM = widgets.FloatSlider(value=4.09e-5, min=1e-10, max=1e-1, step=1e-6,
                             description='Kill rate activated Macrophages(min⁻¹)', style=slider_style, layout=slider_box_layout)

w_M_max = widgets.FloatSlider(value=4, min=1, max=400, step=0.5,
                              description='Max. Macrophages (cells/μl)', style=slider_style, layout=slider_box_layout)
w_NP_max = widgets.FloatSlider(
    value=1.5e4, min=1e3, max=1e5, step=100,
    description='Max. Neutrophils (cells/μl)', style=slider_style, layout=slider_box_layout)

w_EC50_NP = widgets.FloatSlider(value=0.0034, min=0.001, max=0.01, step=0.0001,
                                description='EC50_NP (amount/μl)', style=slider_style, layout=slider_box_layout)

w_T_c_half = widgets.FloatSlider(value=21, min=10, max=50, step=1,
                                 description='Cytokine Half-life (min)', style=slider_style, layout=slider_box_layout)

w_t_delay = widgets.FloatSlider(value=0, min=0, max=100, step=1,
                                description='Immune Delay (min)', style=slider_style, layout=slider_box_layout)

w_ylim_linear = widgets.FloatSlider(value=50, min=50, max=1000, step=10,
                                    description='Y-Axis Max', style=slider_style, layout=slider_box_layout)

w_xlim = widgets.FloatSlider(value=200, min=50, max=800, step=10,
                             description='X-Axis Max', style=slider_style, layout=slider_box_layout)


# Organisiere die Slider in Zeilen:
row1 = widgets.HBox([w_N0, w_k_NP])
row2 = widgets.HBox([w_k_AM, w_M_max])
row3 = widgets.HBox([w_EC50_NP, w_T_c_half])
row4 = widgets.HBox([w_t_delay, w_ylim_linear])
row5 = widgets.HBox([w_NP_max, w_xlim])
slider_container = widgets.VBox([row1, row2, row3, row4, row5],
                                layout=widgets.Layout(margin='20px 0 0 0'))

# Erstelle ein Dictionary der Parameter:
params = {
    'N0': w_N0,
    'k_NP': w_k_NP,
    'k_AM': w_k_AM,
    'M_max': w_M_max,
    'EC50_NP': w_EC50_NP,
    'T_c_half': w_T_c_half,
    't_delay': w_t_delay,
    'ylim_linear': w_ylim_linear,
    'xlim': w_xlim,
    'NP_max': w_NP_max
}


# Verwende interactive_output für die Aktualisierung der Plots:
out_plots = widgets.interactive_output(run_simulation, params)

# Packe die Bereiche in einen Container – so erscheint der erste Plot, dann die Slider, dann der zweite Plot.
container = widgets.VBox([out1, slider_container, out2])
display(container)